In [1]:
from orchestration.nersc import NerscClient

In [2]:
client_id_path = "../clientid.txt"
sfapi_key_path = "../sfapi_training.json"

In [ ]:
client = NerscClient(client_id_path, sfapi_key_path)

user = client.user()
user

In [ ]:
home_path = f"/global/homes/{user.name[0]}/{user.name}"
scratch_path = f"/pscratch/sd/{user.name[0]}/{user.name}"

### The job script below will run a simple python program to generate random numbers


In [ ]:
N = 15

job_script = f"""#!/bin/bash

#SBATCH -q debug
#SBATCH -A als
#SBATCH -N 1
#SBATCH -C cpu
#SBATCH -t 00:10:00
#SBATCH -J sfapi-demo
#SBATCH --exclusive
#SBATCH --output={scratch_path}/nerscClient-test/sfapi-demo-%j.out
#SBATCH --error={scratch_path}/nerscClient-test/sfapi-demo-%j.error

module load python
# Prints N random numbers to form a normal disrobution
python -c "import numpy as np; numbers = np.random.normal(size={N}); [print(n) for n in numbers]"
""" 

In [ ]:
print(job_script)

In [ ]:

### Make sure our output folder is there for our data to go to
client.perlmutter.run(f"mkdir -p {scratch_path}/nerscClient-test")
# We can run ls on the directory to see that it was created
[output_dir] = client.perlmutter.ls(f"{scratch_path}/nerscClient-demo", directory=True)


# Check that the directory is there
output_dir.is_dir()

In [ ]:
### Submit the job and wait for the job to complete
job = client.perlmutter.submit_job(job_script)
print(job)
# Let's save the job id to use later 
job_id = job.jobid

print(f"Waiting for job {job_id} to finish!")
# Wait for the job to finish
job.complete()
print("Done!")

In [ ]:
job.state
print(job.state)